In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from account import Binance
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import cvxpy as cp
from utils.logging import get_logger
from utils.data_helper import *

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = "{:,.4f}".format
warnings.filterwarnings('ignore')

In [2]:
logger = get_logger('Arbitrage')
client = Binance().get_client()
exch_info = client.get_exchange_info()
fees = Binance().get_trading_fee()

In [3]:
price = client.get_orderbook_tickers()
df_symbols = pd.DataFrame(exch_info['symbols'])
df_symbols = df_symbols[['symbol', 'status', 'baseAsset', 'quoteAsset']]

df_lot_sizes = pd.DataFrame([dict(symbol=y['symbol'], **next(filter(lambda x: x['filterType'] == 'LOT_SIZE', y['filters']), {})) for y in exch_info['symbols']])[['stepSize']]
df_px_sizes = pd.DataFrame([dict(symbol=y['symbol'], **next(filter(lambda x: x['filterType'] == 'PRICE_FILTER', y['filters']), {})) for y in exch_info['symbols']])[['tickSize']]

df_symbols = pd.concat([df_symbols, df_lot_sizes, df_px_sizes], axis=1)
df_symbols = df_symbols[df_symbols['status'] == 'TRADING']

df_symbols['qty_decimal'] = df_symbols['stepSize'].apply(count_digit)
df_symbols['price_decimal'] = df_symbols['tickSize'].apply(count_digit)

df_price = pd.DataFrame(price)
df_symbols = pd.merge(df_symbols, df_price, how='left', on='symbol', validate='1:1')
df_symbols = pd.merge(df_symbols, fees, how='left', on='symbol', validate='1:1')
df_symbols['makerCommission'] = df_symbols['makerCommission'].fillna(0)
df_symbols['takerCommission'] = df_symbols['takerCommission'].fillna(0)
df_symbols

,symbol,status,baseAsset,quoteAsset,stepSize,tickSize,qty_decimal,price_decimal,bidPrice,bidQty,askPrice,askQty,makerCommission,takerCommission
0,ETHBTC,TRADING,ETH,BTC,0.00010000,0.00001000,4,5,0.03470000,19.97260000,0.03471000,19.42790000,0.0010,0.0010
1,LTCBTC,TRADING,LTC,BTC,0.00100000,0.00000100,3,6,0.00108800,81.00400000,0.00108900,570.07000000,0.0010,0.0010
2,BNBBTC,TRADING,BNB,BTC,0.00100000,0.00000100,3,6,0.00738100,17.23200000,0.00738200,1.48700000,0.0010,0.0010
3,NEOBTC,TRADING,NEO,BTC,0.01000000,0.00000010,2,7,0.00015670,26.94000000,0.00015680,94.90000000,0.0010,0.0010
4,QTUMETH,TRADING,QTUM,ETH,0.10000000,0.00000100,1,6,0.00098300,4.20000000,0.00099000,61.90000000,0.0010,0.0010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,AIXBTUSDC,TRADING,AIXBT,USDC,0.10000000,0.00010000,1,4,0.46790000,302.70000000,0.46810000,2136.50000000,0.0010,0.0009
1331,CGPTUSDT,TRADING,CGPT,USDT,0.10000000,0.00010000,1,4,0.31520000,1358.90000000,0.31540000,3204.60000000,0.0010,0.0010
1332,CGPTUSDC,TRADING,CGPT,USDC,0.10000000,0.00010000,1,4,0.31510000,154.60000000,0.31540000,279.10000000,0.0010,0.0009
1333,COOKIEUSDT,TRADING,COOKIE,USDT,0.10000000,0.00010000,1,4,0.49870000,1666.10000000,0.49900000,193.40000000,0.0010,0.0010


In [4]:
assets = sorted(list(set(df_symbols['baseAsset'].to_list() + df_symbols['quoteAsset'].to_list())))

# quote matrix: matrix[x,y] => convert currency x to y at market price
quote_matrix = np.ones((len(assets), len(assets)))
fee_matrix = np.zeros((len(assets), len(assets)))

for _, row in df_symbols.iterrows():
    baseAsset = row['baseAsset']
    quoteAsset = row['quoteAsset']

    bidPrice = float(row['bidPrice'])
    askPrice = float(row['askPrice'])

    # market orders follows taker fee    
    takerFee = float(row['takerCommission'])

    base_idx = assets.index(baseAsset)
    quote_idx = assets.index(quoteAsset)

    # sell one unit of base currency to buy quote currency
    quote_matrix[base_idx][quote_idx] = bidPrice

    # sell one unti of quote currency to buy base currency
    quote_matrix[quote_idx][base_idx] = 1/askPrice
    takerFee = 0
    fee_matrix[quote_idx][base_idx] = takerFee
    fee_matrix[base_idx][quote_idx] = takerFee

quote_matrix_w_fee = quote_matrix * (1-fee_matrix)
quote_matrix_ln = -1 * np.log(quote_matrix_w_fee)
quote_matrix_ln[quote_matrix_ln == 0] = 100
np.fill_diagonal(quote_matrix_ln, 0)

In [5]:
logger.info(f"BTC -> ETH: {quote_matrix[assets.index('BTC')][assets.index('ETH')]}")
logger.info(f"ETH -> BTC: {quote_matrix[assets.index('ETH')][assets.index('BTC')]}")

logger.info(f"BTC -> ETH: {quote_matrix_w_fee[assets.index('BTC')][assets.index('ETH')]}")
logger.info(f"ETH -> BTC: {quote_matrix_w_fee[assets.index('ETH')][assets.index('BTC')]}")

2025-01-12 02:40:23,444 - Arbitrage - INFO - BTC -> ETH: 28.810141169691732
2025-01-12 02:40:23,445 - Arbitrage - INFO - ETH -> BTC: 0.0347
2025-01-12 02:40:23,445 - Arbitrage - INFO - BTC -> ETH: 28.810141169691732
2025-01-12 02:40:23,446 - Arbitrage - INFO - ETH -> BTC: 0.0347


# Find the optimal paths

In [6]:
# Alternative: Solve Assignment Problem with CVXPY 
X = cp.Variable((len(quote_matrix_ln),len(quote_matrix_ln)), boolean=True)
ones = np.ones((len(quote_matrix_ln),1))

constraints = [X <= 1,
               X >= 0,
               X @ ones == ones,
               X.T @ ones == ones,
               cp.sum(X) - cp.sum(cp.diag(X)) <= 3
]

# Form objective.
obj = cp.Minimize(cp.sum(cp.multiply(X, quote_matrix_ln)))

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve(verbose=True)  # Returns the optimal value.
print ("status:", prob.status)
print ("optimal value", prob.value)

# avoid some weird rounding
opt_path = X.value.round(2)
df_opt_path = pd.DataFrame(opt_path)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jan 12 02:40:25 AM: Your problem has 163216 variables, 5 constraints, and 0 parameters.
(CVXPY) Jan 12 02:40:25 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 12 02:40:25 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 12 02:40:25 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 12 02:40:25 AM: Compiling problem (target solver=SCIPY).
(CVXPY) Jan 12 02:40:25 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

In [7]:
value = opt_path * quote_matrix * (1-fee_matrix)
value = np.prod(value[value != 0])
print(f'PnL: {100*(value-1):.4f}%')
print()

# 1. First identify all arbitrage trades
trades = {}
for i, row in enumerate(opt_path):
    from_idx, to_idx = i, list(row).index(1)    
    from_asset, to_asset = assets[from_idx], assets[to_idx]         
    if from_asset != to_asset:        
        trades[from_asset] = {
            'from_asset': from_asset,
            'to_asset': to_asset,
            'mkt_price': quote_matrix[from_idx][to_idx],
            'fee': fee_matrix[from_idx][to_idx]
        }

# 2. seggregate the arbitrage trades into groups (optimization can returns more than one closed loops)
visited = set()
group_num = 1

for from_asset in trades:        
    if from_asset in visited:
        continue

    node = from_asset
    trade_order = 1

    while trades[node]['to_asset'] != from_asset:
        trades[node]['group'] = group_num
        trades[node]['order'] = trade_order

        # move to next node
        node = trades[node]['to_asset']

        # update status
        visited.add(node)
        trade_order += 1

    else:
        trades[node]['group'] = group_num
        trades[node]['order'] = trade_order

    group_num += 1
    
df_trades = pd.DataFrame(trades.values())
df_trades = df_trades.sort_values(by=['group', 'order'])
df_trades['mkt_price_w_fee'] = df_trades['mkt_price'] * (1-df_trades['fee'])
df_trades['symbol'] = df_trades.apply(lambda x: x['from_asset'] + x['to_asset'] if x['from_asset'] + x['to_asset'] in df_symbols['symbol'].unique() else x['to_asset'] + x['from_asset'], axis=1)
df_trades = pd.merge(df_trades, df_symbols, how='left', on='symbol', validate='1:1')
df_trades['side'] = np.where(df_trades['to_asset'] == df_trades['baseAsset'], 'BUY', 'SELL')
df_trades

PnL: 0.0506%



,from_asset,to_asset,mkt_price,fee,group,order,mkt_price_w_fee,symbol,status,baseAsset,quoteAsset,stepSize,tickSize,qty_decimal,price_decimal,bidPrice,bidQty,askPrice,askQty,makerCommission,takerCommission,side
0,DENT,ETH,0.0000,0.0000,1,1,0.0000,DENTETH,TRADING,DENT,ETH,1.00000000,0.00000001,0,8,0.00000039,4953406.00000000,0.00000040,14616354.00000000,0.0010,0.0010,SELL
1,ETH,USDT,"3,268.3200",0.0000,1,2,"3,268.3200",ETHUSDT,TRADING,ETH,USDT,0.00010000,0.01000000,4,2,3268.32000000,2.45880000,3268.33000000,84.01760000,0.0010,0.0010,SELL
2,USDT,DENT,784.9294,0.0000,1,3,784.9294,DENTUSDT,TRADING,DENT,USDT,1.00000000,0.00000100,0,6,0.00127200,4327988.00000000,0.00127400,2455314.00000000,0.0010,0.0010,BUY


In [8]:
df_pnl = df_trades.groupby(['group']).prod(numeric_only=True)[['mkt_price', 'mkt_price_w_fee']]
df_pnl.columns = ['gross_pnl%', 'net_pnl%']
df_pnl['gross_pnl%'] = (df_pnl['gross_pnl%'] - 1)*100
df_pnl['net_pnl%'] = (df_pnl['net_pnl%'] - 1)*100
df_pnl = df_pnl.sort_values('net_pnl%', ascending=False)
df_pnl

,gross_pnl%,net_pnl%
group,,
1,0.0506,0.0506


# Trade the arbitrage pair(s)
- Trade the pairs with highest pnl
- The arbitrage loop starts with one existing currency in current portfolios

- commission are included in quote quantity?

In [9]:
trade_currency = {
    'USDT': 50,
    'ETH':  0.01539456264,
    'BTC': 0.00052979531,
}

In [10]:
balance = client.get_account()
balance = pd.DataFrame(balance['balances'])
balance['free'] = balance['free'].astype(float)
balance['locked'] = balance['locked'].astype(float)
balance = balance[balance['free'] > 0]
balance

,asset,free,locked
0,BTC,0.0701,0.0112
2,ETH,1.8638,0.0000
11,USDT,"8,762.5470","2,416.7007"
86,ADA,0.0734,0.0000
88,XLM,99.9000,0.0000
197,EUR,0.0938,0.0000
202,TRY,28.5318,0.0000
227,JPY,4.6434,0.0000
239,SOL,4.6596,0.0000
279,LUNA,0.0000,0.0000


In [11]:
# Trade the First Group For Now
trades = df_trades[df_trades['group'] == df_pnl.index[0]]

# reorder the trades to start with one of the trade currency
start_ccys = [x for x in trade_currency if x in trades['from_asset'].to_list()]
if len(start_ccys) == 0:
    logger.info('No trade currency found')
    raise SystemExit()

start_ccy = start_ccys[0]
start_order = trades[trades['from_asset'] == start_ccy].iloc[0]['order']
trades['order'] = (trades['order'] - start_order) % len(trades) + 1
trades = trades.sort_values('order').reset_index(drop=True)

# sanity check
assert trades.iloc[0]['from_asset'] == start_ccy
assert trades.iloc[-1]['to_asset'] == start_ccy
trades

,from_asset,to_asset,mkt_price,fee,group,order,mkt_price_w_fee,symbol,status,baseAsset,quoteAsset,stepSize,tickSize,qty_decimal,price_decimal,bidPrice,bidQty,askPrice,askQty,makerCommission,takerCommission,side
0,USDT,DENT,784.9294,0.0000,1,1,784.9294,DENTUSDT,TRADING,DENT,USDT,1.00000000,0.00000100,0,6,0.00127200,4327988.00000000,0.00127400,2455314.00000000,0.0010,0.0010,BUY
1,DENT,ETH,0.0000,0.0000,1,2,0.0000,DENTETH,TRADING,DENT,ETH,1.00000000,0.00000001,0,8,0.00000039,4953406.00000000,0.00000040,14616354.00000000,0.0010,0.0010,SELL
2,ETH,USDT,"3,268.3200",0.0000,1,3,"3,268.3200",ETHUSDT,TRADING,ETH,USDT,0.00010000,0.01000000,4,2,3268.32000000,2.45880000,3268.33000000,84.01760000,0.0010,0.0010,SELL


In [12]:
trade_orders = []

# execute the trades via market orders
for i, row in trades.iterrows():
    symbol = row['symbol']
    side = row['side']
    from_asset = row['from_asset']
    to_asset = row['to_asset']    

    baseAsset = row['baseAsset']
    baseAsset_decimal = row['qty_decimal']
    quoteAsset_decimal = row['price_decimal']
    
    # price is from_asset to to_asset, not the quoted price
    mkt_price = float(row['mkt_price'])    

    # First trade are based on pre-set amounts
    # qty is based on from_asset
    if i == 0:            
        from_asset_qty = trade_currency[from_asset]        
    
    order_params = {
        'symbol': symbol,
        'side': side,
        'type':'MARKET'
    }

    if from_asset == baseAsset:
        order_params['quantity'] = round_down(from_asset_qty, baseAsset_decimal)
    else:
        order_params['quoteOrderQty'] = round_down(from_asset_qty, quoteAsset_decimal)
    
    order = client.create_order(**order_params)                    
    logger.info(f'Created Market Order: {order_params}')
    
    if order['status'] == 'FILLED':   

        # find executed quantity for to_asset with current trade, which is from_asset for next trade     
        from_asset_qty = 0
        for fill in order['fills']:
            if to_asset == baseAsset:
                from_asset_qty += float(fill['qty'])
            else:
                from_asset_qty += float(fill['qty']) * float(fill['price'])

            # subtract out comission to get net received quantity
            if 'commission' in fill and to_asset == fill['commissionAsset']:
                from_asset_qty -= float(fill['commission'])

        trade_orders.append(order)
        
    else:
        raise Exception('order not filled yet!')    

2025-01-12 02:40:47,164 - Arbitrage - INFO - Created Market Order: {'symbol': 'DENTUSDT', 'side': 'BUY', 'type': 'MARKET', 'quoteOrderQty': 50.0}
2025-01-12 02:40:47,279 - Arbitrage - INFO - Created Market Order: {'symbol': 'DENTETH', 'side': 'SELL', 'type': 'MARKET', 'quantity': 39206.0}
2025-01-12 02:40:47,481 - Arbitrage - INFO - Created Market Order: {'symbol': 'ETHUSDT', 'side': 'SELL', 'type': 'MARKET', 'quantity': 0.0148}


In [13]:
df_orders = pd.DataFrame(trade_orders).drop(columns=['fills'])
df_orders = df_orders.rename(columns={'side': 'order_side'})

df_fills = df_fills = pd.DataFrame([dict({'symbol': x['symbol']},**y) for x in trade_orders for y in x['fills']])
df_fills = df_fills.rename(columns={'price': 'fill_price', 'qty': 'fill_qty'})
df_fills['fill_price'] = df_fills['fill_price'].astype(float)
df_fills['fill_qty'] = df_fills['fill_qty'].astype(float)
df_fills['commission'] = df_fills['commission'].astype(float)

df_orders = pd.merge(df_orders, df_fills, how='left', on=['symbol'], validate='1:m')
df_orders = df_orders.rename(columns={'status': 'fill_status'})
df_orders = pd.merge(trades, df_orders, how='left', on=['symbol'], validate='1:m')

df_orders['from_asset_qty'] = df_orders.apply(lambda x: x['fill_qty'] if x['from_asset'] == x['baseAsset'] else x['fill_qty'] * x['fill_price'], axis=1)
df_orders['to_asset_gross_qty'] = df_orders.apply(lambda x: x['fill_qty'] if x['to_asset'] == x['baseAsset'] else x['fill_qty'] * x['fill_price'], axis=1)
df_orders['to_asset_comms_qty'] = df_orders.apply(lambda x: x['commission'] if x['to_asset'] == x['commissionAsset'] else 0, axis=1)
df_orders['to_asset_qty'] = df_orders['to_asset_gross_qty'] + df_orders['to_asset_comms_qty']
df_orders

,from_asset,to_asset,mkt_price,fee,group,order,mkt_price_w_fee,symbol,status,baseAsset,quoteAsset,stepSize,tickSize,qty_decimal,price_decimal,bidPrice,bidQty,askPrice,askQty,makerCommission,takerCommission,side,orderId,orderListId,clientOrderId,transactTime,price,origQty,executedQty,origQuoteOrderQty,cummulativeQuoteQty,fill_status,timeInForce,type,order_side,workingTime,selfTradePreventionMode,fill_price,fill_qty,commission,commissionAsset,tradeId,from_asset_qty,to_asset_gross_qty,to_asset_comms_qty,to_asset_qty
0,USDT,DENT,784.9294,0.0000,1,1,784.9294,DENTUSDT,TRADING,DENT,USDT,1.00000000,0.00000100,0,6,0.00127200,4327988.00000000,0.00127400,2455314.00000000,0.0010,0.0010,BUY,879587216,-1,0Sd5As75OW7ftlHkBLXfJy,1736620847172,0.00000000,39246.00000000,39246.00000000,50.00000000,49.99940400,FILLED,GTC,MARKET,BUY,1736620847172,EXPIRE_MAKER,0.0013,"11,015.0000",11.0150,DENT,82309883,14.0331,"11,015.0000",11.0150,"11,026.0150"
1,USDT,DENT,784.9294,0.0000,1,1,784.9294,DENTUSDT,TRADING,DENT,USDT,1.00000000,0.00000100,0,6,0.00127200,4327988.00000000,0.00127400,2455314.00000000,0.0010,0.0010,BUY,879587216,-1,0Sd5As75OW7ftlHkBLXfJy,1736620847172,0.00000000,39246.00000000,39246.00000000,50.00000000,49.99940400,FILLED,GTC,MARKET,BUY,1736620847172,EXPIRE_MAKER,0.0013,"28,231.0000",28.2310,DENT,82309884,35.9663,"28,231.0000",28.2310,"28,259.2310"
2,DENT,ETH,0.0000,0.0000,1,2,0.0000,DENTETH,TRADING,DENT,ETH,1.00000000,0.00000001,0,8,0.00000039,4953406.00000000,0.00000040,14616354.00000000,0.0010,0.0010,SELL,80902951,-1,oBvwaEkfMTGBo3SvNJCKSU,1736620847255,0.00000000,39206.00000000,39206.00000000,0.00000000,0.01489828,FILLED,GTC,MARKET,SELL,1736620847255,EXPIRE_MAKER,0.0000,"39,206.0000",0.0000,ETH,7735913,"39,206.0000",0.0149,0.0000,0.0149
3,ETH,USDT,"3,268.3200",0.0000,1,3,"3,268.3200",ETHUSDT,TRADING,ETH,USDT,0.00010000,0.01000000,4,2,3268.32000000,2.45880000,3268.33000000,84.01760000,0.0010,0.0010,SELL,23561677040,-1,bnk2pvCr972f9JSXl5lJoA,1736620847417,0.00000000,0.01480000,0.01480000,0.00000000,48.36640000,FILLED,GTC,MARKET,SELL,1736620847417,EXPIRE_MAKER,"3,268.0000",0.0148,0.0484,USDT,2036798668,0.0148,48.3664,0.0484,48.4148


In [14]:
df_orders_agg = df_orders.groupby(['group', 'order', 'from_asset']).sum(numeric_only=True).reset_index()
df_orders_agg = df_orders_agg.groupby(['group']).agg({'from_asset': 'first', 'from_asset_qty': 'first', 'to_asset_qty': 'last'})
df_orders_agg['net_qty'] = df_orders_agg['to_asset_qty'] - df_orders_agg['from_asset_qty']
df_orders_agg

,from_asset,from_asset_qty,to_asset_qty,net_qty
group,,,,
1,USDT,49.9994,48.4148,-1.5846
